In [ ]:
import os
import re
import json

### Compare struct_typ with exisiing HAZUS mapping

In [ ]:
# Define the directory containing the JSON files
fragility_folder = "./Dylan_fragilities/FragilityJsons"
hazard_type = "TS" # or "EQ"

# Define a regex pattern to extract structural types
filename_pattern = re.compile(f"LocalCurveSet-{hazard_type}-(.*?)-(High|Mod|Low)Code\.json")

# Set to store unique structural types
structural_types = set()

# Loop through all files in the directory
for filename in os.listdir(fragility_folder):
    match = filename_pattern.match(filename)
    if match:
        structural_type = match.group(1)
        structural_types.add(structural_type)

# Print all unique structural types
print("Unique Structural Types:")
for st in sorted(structural_types):
    print(st)

In [ ]:
from pyincore import FragilityService, IncoreClient

hazus_mapping_id_1 = "5b47b2d9337d4a36187c7563" # Hazus Building Fragility Mapping
hazus_mapping_id_2 = "5b47b350337d4a3629076f2c" # Default Building Fragility Mapping 1.0

client = IncoreClient()

# Initialize FragilityService
fragility_service = FragilityService(client)

# Fetch fragility mappings
hazus_mapping_1 = fragility_service.get_mapping(hazus_mapping_id_1)
hazus_mapping_2 = fragility_service.get_mapping(hazus_mapping_id_2)

In [ ]:
# Function to extract struct_typ values from mapping rules
def extract_structural_types_from_mapping(mapping):
    mapping_types = set()
    
    for mapping_entry in mapping.get("mappings", []):  # Iterate through mapping list
        for rule_group in mapping_entry.get("rules", []):  # "rules" is a list of lists
            for rule in rule_group:
                if "struct_typ EQUALS" in rule:
                    struct_type = rule.split("EQUALS")[1].strip()
                    mapping_types.add(struct_type)
    
    return mapping_types

In [ ]:
# Extract structural types from the mapping rules
mapping_1_types = extract_structural_types_from_mapping(hazus_mapping_1)
mapping_2_types = extract_structural_types_from_mapping(hazus_mapping_2)

# Compare structural types
missing_from_mapping_1 = structural_types - mapping_1_types
missing_from_mapping_2 = structural_types - mapping_2_types

print("\nResults:")
print("Hazus Mapping 1:")
if missing_from_mapping_1:
    print(" - Missing structural types:", missing_from_mapping_1)
else:
    print(" - Contains all structural types.")

print("\nHazus Mapping 2:")
if missing_from_mapping_2:
    print(" - Missing structural types:", missing_from_mapping_2)
else:
    print(" - Contains all structural types.")

### Find overlapping existing fragilities next

In [ ]:
# Dictionary to store local fragility curve expressions
local_fragility_curves = {}

# Iterate over JSON files in the directory
for filename in os.listdir(fragility_folder):
    match = filename_pattern.match(filename)
    if match:
        structural_type = match.group(1)
        code_level = match.group(2)
        file_path = os.path.join(fragility_folder, filename)
        
        with open(file_path, 'r') as file:
            data = json.load(file)
            
            # Extract all fragility curve expressions
            expressions = []
            for fragility_curve in data.get('fragilityCurves', []):
                for rule in fragility_curve.get('rules', []):
                    if 'expression' in rule and rule['expression']:
                        expressions.append(rule['expression'])
            
            # Store extracted expressions with the structural type and code level as key
            local_fragility_curves[f"{structural_type}-{code_level}"] = expressions

In [ ]:
# Search for fragilities related to "HAZUS MH 2.1"
hazus_fragilities = fragility_service.search_dfr3_sets("HAZUS MH 2.1")
# Filter results to only include those belonging to the 'incore' space
hazus_fragilities = [
    fragility for fragility in hazus_fragilities if "incore" in fragility.get("spaces", [])
]

# Extract relevant information from HAZUS fragilities
hazus_fragility_curves = {}
for hazus_fragility in hazus_fragilities:
    description = hazus_fragility.get("description", "")
    if description:
        # Normalize HAZUS descriptions (e.g., "W1 – high code" → "W1-High")
        match = re.search(r"([A-Za-z0-9]+)\s*–\s*(high|mod|low) code", description, re.IGNORECASE)
        if match:
            structural_type = match.group(1).strip().upper()
            code_level = match.group(2).capitalize()
            key = f"{structural_type}-{code_level}"
           
            # Extract all fragility curve expressions
            expressions = []
            for fragility_curve in hazus_fragility.get('fragilityCurves', []):
                for rule in fragility_curve.get('rules', []):
                    if 'expression' in rule and rule['expression']:
                        expressions.append(rule['expression'])
        
            # Store extracted expressions with the structural type and code level as key
            hazus_fragility_curves[f"{structural_type}-{code_level}"] = expressions

In [ ]:
hazus_fragility_ids = [fragility.get("id") for fragility in hazus_fragilities]
hazus_fragility_ids

In [ ]:
hazus_fragility

In [ ]:
# Compare local and HAZUS fragility curves
print("\nComparison of Local vs. HAZUS Fragility Curves:")
for key, local_expressions in local_fragility_curves.items():
    hazus_expressions = hazus_fragility_curves.get(key, [])
    
    if hazus_expressions:
        print(f"Match found for {key}")
        
        # Check if expressions match
        if set(local_expressions) == set(hazus_expressions):
            print("  ✅ Expressions Match!")
        else:
            print("  ❌ Expressions Differ!")

### Posting fragilites to INCORE

In [ ]:
# Define a pattern to extract structural type and design level from filenames
filename_pattern = re.compile(f"LocalCurveSet-{hazard_type}-(.*?)-(High|Mod|Low)Code\.json")

# Dictionary to store posted fragility IDs
fragility_id_mapping = {}

# Iterate over JSON files and post them to IN-CORE
for filename in os.listdir(fragility_folder):
    match = filename_pattern.match(filename)
    if match:
        structural_type = match.group(1).strip().upper()
        design_level = match.group(2).capitalize()
        description = f"{structural_type} - {design_level} Code"

        file_path = os.path.join(fragility_folder, filename)
        with open(file_path, 'r') as file:
            fragility_data = json.load(file)

            # Remove ID if exists
            fragility_data.pop("id", None)

            # Recursively replace "momentumflux" with "Mmax" in the entire JSON structure
            def replace_momentumflux(data):
                if isinstance(data, dict):
                    return {key: replace_momentumflux(value) for key, value in data.items()}
                elif isinstance(data, list):
                    return [replace_momentumflux(item) for item in data]
                elif isinstance(data, str) and "momentumflux" in data.lower():
                    return data.lower().replace("momentumflux", "Mmax")
                return data

            fragility_data = replace_momentumflux(fragility_data)

            # Update description and metadata
            fragility_data["description"] = description
            fragility_data["authors"] = ["HAZUS MH 2.1"]

            # Post to IN-CORE
            posted_fragility = fragility_service.create_dfr3_set(fragility_data)

            # Store the newly created fragility ID
            fragility_id_mapping[f"{structural_type}-{design_level}"] = posted_fragility["id"]

In [ ]:
fragility_id_mapping

In [ ]:
# Load and modify fragility mapping
mapping_path = f"./Dylan_fragilities/{hazard_type}MappingSet.json"
with open(mapping_path, 'r') as file:
    mapping_data = json.load(file)

# Update mapping name
if hazard_type == "EQ":
    mapping_data["name"] = "HAZUS 2.1 Earthquake Building Fragility Mapping (NSI Data)"
elif hazard_type == "TS":
    mapping_data["name"] = "HAZUS 2.1 Tsunami Building Fragility Mapping (NSI Data)"

# Remove ID if exists
mapping_data.pop("id", None)

# Replace fragility IDs in mapping
for entry in mapping_data.get("mappings", []):
    if hazard_type == "EQ":
        entry_key = "Non-Retrofit Fragility ID Code"
    elif hazard_type == "TS":
        entry_key = "Non-Retrofit MomentumFlux Fragility ID Code"
    fragility_filename = entry["entry"].get(entry_key)
    
    # Extract structural type and design level from filename
    match = filename_pattern.match(fragility_filename)
    if match:
        structural_type = match.group(1).strip().upper()
        design_level = match.group(2).capitalize()
        key = f"{structural_type}-{design_level}"

        # Replace with posted fragility ID
        if key in fragility_id_mapping:
            entry["entry"][entry_key] = fragility_id_mapping[key]

# Post updated mapping to IN-CORE
posted_mapping = fragility_service.create_mapping(mapping_data)

# Display posted mapping ID
print(f"Posted Mapping ID: {posted_mapping['id']}")

In [ ]:
# add to space
from pyincore import SpaceService

spacesvc = SpaceService(client)

space_name = "incore"

# Add the posted fragility to the IN-CORE space
for key, fragility_id in fragility_id_mapping.items():
    space = spacesvc.add_to_space_by_name(space_name, fragility_id)
    assert fragility_id in space["members"]  # Ensure it was successfully added

# add mapping id
space = spacesvc.add_to_space_by_name(space_name, posted_mapping["id"])
assert fragility_id in space["members"]  # Ensure it was successfully added